In [1]:
load('packaging.rda', verbose=T)

Loading objects:
  save_functions
  set_parallel


### Seurat data handling

#### Group cells using variables

In [2]:
get.grouped.cells <- function(object, group.vars){
    grouped.cells <- split(
        colnames(object), 
        apply(object@meta.data[, group.vars], 1, paste, collapse='_')
    )
    return(grouped.cells)
}

#### Merge seurat object list

In [3]:
merge.seurat <- function(seurat_list, merge.data = T){
    return(
        purrr::reduce(seurat_list,
                      function(x, y) {merge(x = x, y = y, merge.data = merge.data)})
        )
}

#### Rename cells

In [4]:
rename.seurat.cells <- function(object, new.names, assays=c('RNA'), slots=c('counts', 'data', 'scale.data')){
    
    require(Seurat)
    
    meta.data <- object@meta.data
    rownames(meta.data) <- new.names
    
    counts <- object[['RNA']]@counts
    colnames(counts) <- new.names
    
    newobj <- CreateSeuratObject(counts = counts, row.names = rownames(object), min.cells = 0, min.features = 0, assay = 'RNA', meta.data = meta.data)
    
    for (assay in names(object)){
        if (!assay%in%assays){next()}
        for (sl in slotNames(object[[assay]])){
            if (!sl%in%slots){next()}
            
            slotdata <- slot(object[[assay]], sl)
            if (0%in%dim(slotdata)){next()}
            
            colnames(slotdata) <- new.names
            slot(newobj[[assay]], sl) <- slotdata
        }
    }
    return(newobj)
}

#### Subset Seurat from (maybe duplicated) cells

In [5]:
get.seurat.from.cells <- function(object, cells){
    require(dplyr)
    
    if(!any(duplicated(cells))){
        return(object[,cells])
    } else{
        cells <- data.frame(cells)%>%group_by(cells)%>%
            mutate(rep=1:length(cells), new.cell=paste(cells, rep, sep='_'))

        new.obj <- lapply(
            split(cells, cells$rep), function(celldata){
                new.obj <- object[,celldata$cells]
                new.obj <- rename.seurat.cells(object = new.obj, new.names = celldata$new.cell)
                new.obj
            }) %>% merge.seurat(merge.data = T) 
        return(new.obj)
    }
}

#### Dimensionality reduction automatic

In [6]:
dimred.auto <- function(object, lognorm=T, scale.factor=10000, 
                        plot.vars=NULL, base.size = 5, umap=T, tsne=T, pca=T){
    
    require(Seurat)
    
    suppressMessages(suppressWarnings(expr = {
        
        if (lognorm){
            object <- NormalizeData(object, scale.factor=scale.factor)
        }
        object <- ScaleData(object)

        object <- FindVariableFeatures(object)

        npcs <- min(ncol(object)-1, 50)
        object <- RunPCA(object, npcs = npcs)


        options(repr.plot.height=8, repr.plot.width=16)


        
    }))
                     
    if (!is.null(plot.vars) & pca){
        
        pca.plots <- NULL
        for (pvar in plot.vars){
            p <- LabelClusters(DimPlot(object, pt.size = 4, reduction = 'pca',
                                   group.by = pvar,# split.by='stim'
                                  ), id=pvar)
            pca.plots <- if(is.null(pca.plots)){p}else{pca.plots+p}
        }
        plot(pca.plots)
        
    }
    if (umap){
        object <- RunUMAP(object, dims=1:npcs, verbose=F, spread = 4, min.dist = 0)
        if (!is.null(plot.vars)){

            umap.plots <- NULL
            for (pvar in plot.vars){
                p <- LabelClusters(DimPlot(object, pt.size = 4, reduction = 'umap',
                                       group.by = pvar,# split.by='stim'
                                      ), id=pvar)
                umap.plots <- if(is.null(umap.plots)){p}else{umap.plots+p}
            }
            plot(umap.plots)
        }
    }

    if (tsne){
        object <- RunTSNE(object, dims=1:npcs, perplexity=5)

        if (!is.null(plot.vars)){

            tsne.plots <- NULL
            for (pvar in plot.vars){
                p <- LabelClusters(DimPlot(object, pt.size = 4, reduction = 'tsne',
                                       group.by = pvar,# split.by='stim'
                                      ), id=pvar)
                tsne.plots <- if(is.null(tsne.plots)){p}else{tsne.plots+p}
            }
            plot(tsne.plots)
        }
    }

    return(object)
}

In [7]:
save_functions('seurat_handling')

Loading required package: glue



[1] "dimred.auto <- function (object, lognorm = T, scale.factor = 10000, plot.vars = NULL, \n    base.size = 5, umap = T, tsne = T, pca = T) \n{\n    require(Seurat)\n    suppressMessages(suppressWarnings(expr = {\n        if (lognorm) {\n            object <- NormalizeData(object, scale.factor = scale.factor)\n        }\n        object <- ScaleData(object)\n        object <- FindVariableFeatures(object)\n        npcs <- min(ncol(object) - 1, 50)\n        object <- RunPCA(object, npcs = npcs)\n        options(repr.plot.height = 8, repr.plot.width = 16)\n    }))\n    if (!is.null(plot.vars) & pca) {\n        pca.plots <- NULL\n        for (pvar in plot.vars) {\n            p <- LabelClusters(DimPlot(object, pt.size = 4, reduction = \"pca\", \n                group.by = pvar, ), id = pvar)\n            pca.plots <- if (is.null(pca.plots)) {\n                p\n            }\n            else {\n                pca.plots + p\n            }\n        }\n        plot(pca.plots)\n    }\n    if (umap) {\n        object <- RunUMAP(object, dims = 1:npcs, verbose = F, \n            spread = 4, min.dist = 0)\n        if (!is.null(plot.vars)) {\n            umap.plots <- NULL\n            for (pvar in plot.vars) {\n                p <- LabelClusters(DimPlot(object, pt.size = 4, \n                  reduction = \"umap\", group.by = pvar, ), id = pvar)\n                umap.plots <- if (is.null(umap.plots)) {\n                  p\n                }\n                else {\n                  umap.plots + p\n                }\n            }\n            plot(umap.plots)\n        }\n    }\n    if (tsne) {\n        object <- RunTSNE(object, dims = 1:npcs, perplexity = 5)\n        if (!is.null(plot.vars)) {\n            tsne.plots <- NULL\n            for (pvar in plot.vars) {\n                p <- LabelClusters(DimPlot(object, pt.size = 4, \n                  reduction = \"tsne\", group.by = pvar, ), id = pvar)\n                tsne.plots <- if (is.null(tsne.plots)) {\n                  p\n                }\n                else {\n                  tsne.plots + p\n                }\n            }\n            plot(tsne.plots)\n        }\n    }\n    return(object)\n}\n\nget.grouped.cells <- function (object, group.vars) \n{\n    grouped.cells <- split(colnames(object), apply(object@meta.data[, \n        group.vars], 1, paste, collapse = \"_\"))\n    return(grouped.cells)\n}\n\nget.seurat.from.cells <- function (object, cells) \n{\n    require(dplyr)\n    if (!any(duplicated(cells))) {\n        return(object[, cells])\n    }\n    else {\n        cells <- data.frame(cells) %>% group_by(cells) %>% mutate(rep = 1:length(cells), \n            new.cell = paste(cells, rep, sep = \"_\"))\n        new.obj <- lapply(split(cells, cells$rep), function(celldata) {\n            new.obj <- object[, celldata$cells]\n            new.obj <- rename.seurat.cells(object = new.obj, \n                new.names = celldata$new.cell)\n            new.obj\n        }) %>% merge.seurat(merge.data = T)\n        return(new.obj)\n    }\n}\n\nmerge.seurat <- function (seurat_list, merge.data = T) \n{\n    return(purrr::reduce(seurat_list, function(x, y) {\n        merge(x = x, y = y, merge.data = merge.data)\n    }))\n}\n\nrename.seurat.cells <- function (object, new.names, assays = c(\"RNA\"), slots = c(\"counts\", \n    \"data\", \"scale.data\")) \n{\n    require(Seurat)\n    meta.data <- object@meta.data\n    rownames(meta.data) <- new.names\n    counts <- object[[\"RNA\"]]@counts\n    colnames(counts) <- new.names\n    newobj <- CreateSeuratObject(counts = counts, row.names = rownames(object), \n        min.cells = 0, min.features = 0, assay = \"RNA\", meta.data = meta.data)\n    for (assay in names(object)) {\n        if (!assay %in% assays) {\n            (next)()\n        }\n        for (sl in slotNames(object[[assay]])) {\n            if (!sl %in% slots) {\n                (next)()\n            }\n            slotdata <- slot(object[[assay]], sl)\n            if (0 %in% 